In [27]:
import numpy as np
import math
import copy

"""
This is the dimension of the Lynx Robot stated as global variable

"""
# Lynx ADL5 constants in mm
d1 = 76.2                      # Distance between joint 1 and joint 2
a2 = 146.05                    # Distance between joint 2 and joint 3
a3 = 187.325                   # Distance between joint 3 and joint 4
d4 = 34                        # Distance between joint 4 and joint 5
d5 = 68                        # Distance between joint 4 and end effector

# Joint limits
lowerLim = np.array([-1.4, -1.2, -1.8, -1.9, -2.0, -15]).reshape((1, 6))    # Lower joint limits in radians (grip in mm (negative closes more firmly))
upperLim = np.array([1.4, 1.4, 1.7, 1.7, 1.5, 30]).reshape((1, 6))          # Upper joint limits in radians (grip in mm)

def inverse(T0e):
    """
    INPUT:
    T - 4 x 4 homogeneous transformation matrix, representing
       the end effector frame expressed in the base (0) frame
       (position in mm)

    OUTPUT:
    q - a n x 5 vector of joint inputs [q1,q2,q3,q4,q5] (rad)
       which are required for the Lynx robot to reach the given
       transformation matrix T. Each row represents a single
       solution to the IK problem. If the transform is
       infeasible, q should be all zeros.
    isPos - a boolean set to true if the provided
         transformation T is achievable by the Lynx robot as given,
         ignoring joint limits
    """
    isPos = 1
    q = np.zeros((1, 6))
    # Your code starts from here

    # End effector position
    e_pos = T0e[0:3,3]

    # Wrist position
    wrist_pos = e_pos - d5*T0e[0:3,2]
    print("wrist position: ", wrist_pos)
    print("end_effector position: ", e_pos)

    # Theta_1
    # First and Fourth quadrant
    theta1 = np.arctan(wrist_pos[1] / wrist_pos[0])
    print('Theta_1 = ', theta1, ' rads')
    q[0,0] = np.around(theta1, 5)

    # Second quadrant and Third quadrant yield the same solutions
    # as First and Fourth

    # Check for feasible orientation of end-effector
    theta_check = np.arctan(e_pos[1] / e_pos[0])
    if(abs(theta_check-theta1) <  0.000001):
        isPos = True
    else: isPos = False

    # Theta_3-- first solution
    theta3 = np.pi/2 -np.arccos((wrist_pos[0]**2 + wrist_pos[1]**2 + (wrist_pos[2] - d1)**2 - a2**2 - a3**2) / (2*a2*a3))
    q[0,2] = np.around(theta3, 5)
    print('Theta_3 = ', q[0,2], ' rads')

    # Theta_3-- second solution
    temp = np.arccos((wrist_pos[0]**2 + wrist_pos[1]**2 + (wrist_pos[2] - d1)**2 - a2**2 - a3**2) / (2*a2*a3))
    if(temp != None):
        temp += -3*np.pi/2
        new_row = np.zeros((1,6))
        new_row[0,0] = theta1
        new_row[0,2] = temp
        q = np.concatenate((q, new_row))
        print('Theta_3 = ', temp, ' rads')

    # Theta_2
    for i in range(q.shape[0]):
        theta3 = q[i, 2]
        theta2 =  np.pi/2 - np.arctan2((wrist_pos[2] - d1) , (np.sqrt(wrist_pos[0]**2 + wrist_pos[1]**2))) + np.arctan2((a3*np.sin(-np.pi/2 - theta3)) , (a2 + a3*np.cos(-np.pi/2 - theta3)))
        q[i,1] = np.around(theta2, 5)
        print('Theta_2 = ', q[i,1], ' rads')

    #print('q before populating every theta: ', q)

    # row index of q array for rotational matrices to reference
    i = 0

    # Rotation matrix from frame 0 to frame 1
    R_01 = np.zeros((3, 3))
    R_01[2,1]= -1
    R_01[1,2]= np.cos(q[i,0])
    R_01[1,0]= np.sin(q[i,0])
    R_01[0,2]= -np.sin(q[i,0])
    R_01[0,0]= np.cos(q[i,0])

    # Rotation matrix from frame 1 to frame 2
    R_12 = np.zeros((3, 3))
    R_12[2,2]= 1
    R_12[1,1]= np.cos(q[i,1]-np.pi/2)
    R_12[1,0]= np.sin(q[i,1]-np.pi/2)
    R_12[0,1]= -np.sin(q[i,1]-np.pi/2)
    R_12[0,0]= np.cos(q[i,1]-np.pi/2)

    # Rotation matrix from frame 2 to frame 3
    R_23 = np.zeros((3, 3))
    R_23[2,2]= 1
    R_23[1,1]= np.cos(q[i,2]+np.pi/2)
    R_23[1,0]= np.sin(q[i,2]+np.pi/2)
    R_23[0,1]= -np.sin(q[i,2]+np.pi/2)
    R_23[0,0]= np.cos(q[i,2]+np.pi/2)

    # Rotation matrix from frame 3 to frame 4
    R_34 = np.zeros((3, 3))
    R_34[2,1] = -1
    R_34[1,2] = np.cos(q[i,3]-np.pi/2)
    R_34[1,0] = np.sin(q[i,3]-np.pi/2)
    R_34[0,2] = -np.sin(q[i,3]-np.pi/2)
    R_34[0,0] = np.cos(q[i,3]-np.pi/2)

    # Rotation matrix from frame 4 to frame 5
    R_45 = np.zeros((3, 3))
    R_45[2,2] = 1
    R_45[1,1] = np.cos(q[i,4])
    R_45[1,0] = np.sin(q[i,4])
    R_45[0,1] = -np.sin(q[i,4])
    R_45[0,0] = np.cos(q[i,4])

    # Rotation from frame 0 to end effector
    R = np.zeros((3, 3))
    for i in range(0, 3):
        for j in range(0, 3):
            R[i,j] = T0e[i,j]

    #print('R_0e = ', R)

    if(isPos):
        for row_idx in range(q.shape[0]):
            # Which row from q to reference for matrix angles
            #print("Calculating theta4 and theta5 for row ", i, " of ", q.shape[0])
            i = row_idx

            R_03 = np.matmul(np.matmul(R_01, R_12), R_23)
            #print("R_03 ")
            #print(R_03)

            R_3e = np.matmul(np.transpose(R_03) , R)
            #print("R_3e ")
            #print(R_3e)

            theta5 = np.arctan2(-R_3e[2,0] , -R_3e[2,1])
            theta4 = np.arctan2(-R_3e[0,2] , R_3e[1,2]) + np.pi/2

            print('Theta_4 = ', theta4, ' rads')
            print('Theta_5 = ', theta5, ' rads')

            q[i,3] = theta4
            q[i,4] = theta5

            R_check = np.matmul(np.matmul(R_03, R_34), R_45)
            #print("R_check", R_check)

        print('q after populating every theta: ', q)

    # Account for joint limits
    exceeds_limits = [False for row in q]

    for row_idx in range(q.shape[0]):
        for theta_idx in range(q.shape[1]):
            if (q[row_idx, theta_idx] > upperLim[0, theta_idx]) or (q[row_idx, theta_idx] < lowerLim[0, theta_idx]):
                exceeds_limits[row_idx] = (exceeds_limits[row_idx] | True)

    print("Joint limits exceeded: ", exceeds_limits)
    # q = q[np.invert(exceeds_limits),:]
    # print("q after filtering: ", q)

    # Your code ends here

    return q, isPos


In [28]:
# T0e=np.array([[   0.99500416,    0.09983344,    0.00000973,   -9.99981939],
#        [  -0.09983344,    0.99500416,    0.00008253, -375.99841183],
#        [  -0.00000144,   -0.00008309,    1.        ,   29.00170962],
#        [   0.        ,    0.        ,    0.        ,    1.        ]])
T0e=np.array([[0,0,1, 255.325],
       [0,-1,0,0],
       [1,0,0,222.25],
       [0,0,0,1]])
inverse(T0e)

wrist position:  [187.325   0.    222.25 ]
end_effector position:  [255.325   0.    222.25 ]
Theta_1 =  0.0  rads
Theta_3 =  -0.0  rads
Theta_3 =  -3.141592653589793  rads
Theta_2 =  0.0  rads
Theta_2 =  1.81716  rads
Theta_4 =  0.0  rads
Theta_5 =  -0.0  rads
Theta_4 =  0.0  rads
Theta_5 =  -0.0  rads
q after populating every theta:  [[ 0.          0.         -0.          0.         -0.          0.        ]
 [ 0.          1.81716    -3.14159265  0.         -0.          0.        ]]
Joint limits exceeded:  [False, True]


(array([[ 0.        ,  0.        , -0.        ,  0.        , -0.        ,
          0.        ],
        [ 0.        ,  1.81716   , -3.14159265,  0.        , -0.        ,
          0.        ]]),
 True)